In [1]:
import rasterio
import numpy as np
file_name = '/work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Biarritz/Biarritz_EPSG32630_1.tif'
with rasterio.open(file_name) as src:
    band1 = src.read(1)
    print('Band1 has shape', band1.shape)
    height = band1.shape[0]
    width = band1.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(src.transform, rows, cols)
    lons= np.array(xs)
    lats = np.array(ys)
    print('lons shape', lons.shape)

Band1 has shape (2048, 2048)
lons shape (2048, 2048)


In [3]:
print(cols)

[[   0    1    2 ... 2045 2046 2047]
 [   0    1    2 ... 2045 2046 2047]
 [   0    1    2 ... 2045 2046 2047]
 ...
 [   0    1    2 ... 2045 2046 2047]
 [   0    1    2 ... 2045 2046 2047]
 [   0    1    2 ... 2045 2046 2047]]


In [2]:
lons

array([[620734.92797033, 620735.42763938, 620735.92730842, ...,
        621756.75116902, 621757.25083806, 621757.75050711],
       [620734.92797033, 620735.42763938, 620735.92730842, ...,
        621756.75116902, 621757.25083806, 621757.75050711],
       [620734.92797033, 620735.42763938, 620735.92730842, ...,
        621756.75116902, 621757.25083806, 621757.75050711],
       ...,
       [620734.92797033, 620735.42763938, 620735.92730842, ...,
        621756.75116902, 621757.25083806, 621757.75050711],
       [620734.92797033, 620735.42763938, 620735.92730842, ...,
        621756.75116902, 621757.25083806, 621757.75050711],
       [620734.92797033, 620735.42763938, 620735.92730842, ...,
        621756.75116902, 621757.25083806, 621757.75050711]])

In [1]:
import rasterio

def read_window_from_big_raster(window, path, raster_path):
    with rasterio.open(path) as image_file:
        with rasterio.open(raster_path) as raster_file:
            left, bottom, right, top = rasterio.windows.bounds(
                window, 
                transform=image_file.transform
            )
            rw = rasterio.windows.from_bounds(
                left, bottom, right, top, 
                transform=raster_file.transform
            )
            image = raster_file.read(
                window=rw, 
                out_dtype=np.float32
            )
    return image

In [43]:
from rasterio.windows import Window
import numpy as np
window = Window(0, 0, 10000, 10000)
path = p+f'/Biarritz_EPSG32630_9.tif'
big_path = p+'/BIARRITZ_20140902_T_TOA_reproj-EPSG:32630.tif'
img = read_window_from_big_raster(window, path, big_path)

In [44]:
def read_window_basic(window, path):
    with rasterio.open(path) as image_file:
        image = image_file.read(window=window, out_dtype=np.float32)
    return image

In [50]:
window = Window(0, 0, 10000, 10000)
img2 = read_window_basic(window,big_path)

In [51]:
img2.shape

(4, 10000, 10000)

In [1]:
import rasterio
p = '/work/OT/ai4usr/fournip/digitanie/Toulouse/toulouse_full_tiled.tif'
ds = rasterio.open(p)
print(ds)

<open DatasetReader name='/work/OT/ai4usr/fournip/digitanie/Toulouse/toulouse_full_tiled.tif' mode='r'>


In [2]:
a = ds.statistics(bidx=1)

In [3]:
a

Statistics(min=-0.0029133798088878393, max=1.5431257486343384, mean=0.09988372604258908, std=0.05581474629124038)

In [4]:
a.min

-0.0029133798088878393